### Run the level-2b pipeline (```calwebb_image2```)

Documentation: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image2.html#calwebb-image2-stage-2-imaging-processing

In [1]:
from jwst.pipeline import Image2Pipeline

# filename = args.rate_image
filename = 'v1/jw01144030001_02101_00004_nrcblong_cal.fits'


# distortionfile = args.distortion_asdf
distortionfile = '/grp/crds/cache/references/jwst/jwst_nircam_distortion_0133.asdf'

print(f'reducing file {filename} with distortion file {distortionfile}')

stage2_img = Image2Pipeline()
stage2_img.resample.save_results = False
stage2_img.save_results = True
stage2_img.override_distortion = distortionfile

level2_result = stage2_img.run(filename) 

2022-04-29 19:10:26,140 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.
2022-04-29 19:10:26,142 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2022-04-29 19:10:26,144 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2022-04-29 19:10:26,145 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2022-04-29 19:10:26,147 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.
2022-04-29 19:10:26,149 - stpipe.Image2Pipeline.resample - INFO - ResampleStep instance created.
2022-04-29 19:10:26,201 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline running with args ('v1/jw01144030001_02101_00004_nrcblong_cal.fits',).
2022-04-29 19:10:26,206 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, '

reducing file v1/jw01144030001_02101_00004_nrcblong_cal.fits with distortion file /grp/crds/cache/references/jwst/jwst_nircam_distortion_0133.asdf


2022-04-29 19:10:27,494 - stpipe.Image2Pipeline - INFO - Prefetching reference files for dataset: 'jw01144030001_02101_00004_nrcblong_cal.fits' reftypes = ['area', 'camera', 'collimator', 'dflat', 'disperser', 'distortion', 'drizpars', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'ifufore', 'ifupost', 'ifuslicer', 'msa', 'ote', 'photom', 'regions', 'sflat', 'specwcs', 'wavelengthrange', 'wfssbkg']
2022-04-29 19:10:31,131 - stpipe.Image2Pipeline - INFO - Prefetch for AREA reference file is '/grp/crds/cache/references/jwst/jwst_nircam_area_0026.fits'.
2022-04-29 19:10:31,156 - stpipe.Image2Pipeline - INFO - Prefetch for CAMERA reference file is 'N/A'.
2022-04-29 19:10:31,157 - stpipe.Image2Pipeline - INFO - Prefetch for COLLIMATOR reference file is 'N/A'.
2022-04-29 19:10:31,157 - stpipe.Image2Pipeline - INFO - Prefetch for DFLAT reference file is 'N/A'.
2022-04-29 19:10:31,158 - stpipe.Image2Pipeline - INFO - Prefetch for DISPERSER reference file is 'N/A'.
2022-04-29 19:10:31,159 - s

2022-04-29 19:10:47,195 - stpipe.Image2Pipeline.resample - INFO - Drizzling (2048, 2048) --> (2067, 2074)
2022-04-29 19:10:48,638 - stpipe.Image2Pipeline.resample - INFO - Resampling var_poisson
2022-04-29 19:10:50,617 - stpipe.Image2Pipeline.resample - INFO - Drizzling (2048, 2048) --> (2067, 2074)
2022-04-29 19:10:52,020 - stpipe.Image2Pipeline.resample - INFO - Resampling var_flat
2022-04-29 19:10:53,959 - stpipe.Image2Pipeline.resample - INFO - Drizzling (2048, 2048) --> (2067, 2074)
2022-04-29 19:10:55,369 - stpipe.Image2Pipeline.resample - INFO - Update S_REGION to POLYGON ICRS  80.481741540 -69.491103721 80.584990438 -69.492175465 80.587967530 -69.455879412 80.484893019 -69.454809479
2022-04-29 19:10:55,965 - stpipe.Image2Pipeline.resample - INFO - Saved model in jw01144030001_02101_00004_nrcblong_i2d.fits
2022-04-29 19:10:55,965 - stpipe.Image2Pipeline.resample - INFO - Step resample done
2022-04-29 19:10:55,966 - stpipe.Image2Pipeline - INFO - Finished processing product v1/jw

### Run only the alignment step of level-3 pipeline (```tweakreg```, which includes Gaia alignment)

Documentation: https://jwst-pipeline.readthedocs.io/en/latest/jwst/tweakreg/README.html

First, you need to edit the tweakreg code in the local JWST pipeline environment, because it's hardcoded to skip ```tweakreg``` if there is only one input image (it's expecting an association with multiple exposures). If you need help figuring out where the scripts are located, use (thanks, Bryan!):

```python
import jwst, tweakwcs
print(jwst.tweakreg.__file__)
print(tweakwcs.__file__)
```

Installation instructions are here: https://github.com/spacetelescope/jwst#installing-for-developers

Go to: ```/jwst/jwst/tweakreg/tweakreg_step.py``` and you'll see the code starting at **line 136**:

```python
        if len(grp_img) == 1:
            self.log.info("* Images in GROUP 1:")
            for im in grp_img[0]:
                self.log.info("     {}".format(im.meta.filename))
            self.log.info('')

            # we need at least two exposures to perform image alignment
            self.log.warning("At least two exposures are required for image "
                             "alignment.")
            self.log.warning("Nothing to do. Skipping 'TweakRegStep'...")
            self.skip = True
            for model in images:
                model.meta.cal_step.tweakreg = "SKIPPED"
                # Remove the attached catalogs
                del model.catalog
            return input
```
**Comment this whole section out.**



Next, at **line 196**:

```python
except ValueError as e:
    msg = e.args[0]
    if (msg == "Too few input images (or groups of images) with "
            "non-empty catalogs."):
        # we need at least two exposures to perform image alignment
        self.log.warning(msg)
        self.log.warning("At least two exposures are required for "
                         "image alignment.")
        self.log.warning("Nothing to do. Skipping 'TweakRegStep'...")
        self.skip = True
        for model in images:
            model.meta.cal_step.tweakreg = "SKIPPED"
        return images
```    

This section should be changed to: 

```python
except ValueError as e:
    raise e

    # msg = e.args[0]
    # if (msg == "Too few input images (or groups of images) with "
    #         "non-empty catalogs."):
    #     # we need at least two exposures to perform image alignment
    #     self.log.warning(msg)
    #     self.log.warning("At least two exposures are required for "
    #                      "image alignment.")
    #     self.log.warning("Nothing to do. Skipping 'TweakRegStep'...")
    #     self.skip = True
    #     for model in images:
    #         model.meta.cal_step.tweakreg = "SKIPPED"
    #     return images
    #
    # else:
    #     raise e   
```

Now, go to ```tweakwcs/imalign.py``` and **line 601**:

```python
# check that we have enough input images:
if (refcat is None and len(wcs_gcat) < 2) or len(wcs_gcat) == 0:
    raise ValueError("Too few input images (or groups of images) with "
                     "non-empty catalogs.")
```

**Comment this whole section out.**

In [2]:
from jwst.tweakreg import TweakRegStep

You can get all the available step parameters by:

In [3]:
from jwst.tweakreg import TweakRegStep 
print(TweakRegStep.spec)


        save_catalogs = boolean(default=False) # Write out catalogs?
        catalog_format = string(default='ecsv') # Catalog output file format
        kernel_fwhm = float(default=2.5) # Gaussian kernel FWHM in pixels
        snr_threshold = float(default=10.0) # SNR threshold above the bkg
        brightest = integer(default=1000) # Keep top ``brightest`` objects
        peakmax = float(default=None) # Filter out objects with pixel values >= ``peakmax``
        enforce_user_order = boolean(default=False) # Align images in user specified order?
        expand_refcat = boolean(default=False) # Expand reference catalog with new sources?
        minobj = integer(default=15) # Minimum number of objects acceptable for matching
        searchrad = float(default=1.0) # The search radius in arcsec for a match
        use2dhist = boolean(default=True) # Use 2d histogram to find initial offset?
        separation = float(default=0.5) # Minimum object separation in arcsec
        tolerance = f

**Note that you need to use a search radius large enough that when it's divided by 10, sources are still found.**

See https://jira.stsci.edu/browse/JP-2585 for more information.

In [7]:
tweakreg = TweakRegStep()
tweakreg.kernel_fwhm = 2.302   
tweakreg.snr_threshold = 20.0  
tweakreg.brightest = 1000       
tweakreg.save_results = True
tweakreg.searchrad = 2.0
tweakreg.align_to_gaia = True
tweakreg.gaia_catalog = 'GAIADR2'
tweakreg.save_gaia_catalog = True

tweak = tweakreg.run(level2_result)

2022-04-29 19:15:27,356 - stpipe.TweakRegStep - INFO - TweakRegStep instance created.
2022-04-29 19:15:27,478 - stpipe.TweakRegStep - INFO - Step TweakRegStep running with args ([<ImageModel(2048, 2048) from jw01144030001_02101_00004_nrcblong_cal.fits>],).
2022-04-29 19:15:27,481 - stpipe.TweakRegStep - INFO - Step TweakRegStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'save_catalogs': False, 'catalog_format': 'ecsv', 'kernel_fwhm': 2.302, 'snr_threshold': 20.0, 'brightest': 1000, 'peakmax': None, 'enforce_user_order': False, 'expand_refcat': False, 'minobj': 15, 'searchrad': 2.0, 'use2dhist': True, 'separation': 0.5, 'tolerance': 1.0, 'xoffset': 0.0, 'yoffset': 0.0, 'fitgeometry': 'general', 'nclip': 3, 'sigma': 3.0, 'align_to_gaia': True, 'gaia_catalog': 'GAIADR2'

It runs! Check some stuff using Bryan's JWebbinar notebook snippets.

In [10]:
cal_file = 'jw01144001001_02101_00005_nrcalong_cal.fits'
tweak_file = tweak[0].meta.filename

In [11]:
from jwst import datamodels

# Open the files using datamodels
cal_data = datamodels.open(cal_file)
tweak_data = datamodels.open(tweak_file)

In [12]:
cal_detector_to_world = cal_data.meta.wcs.get_transform('detector', 'world')
run_detector_to_world = tweak_data.meta.wcs.get_transform('detector', 'world')

Let's look at RA, Dec in the center of the detector.

In [13]:
x, y = (1024, 1024)

In [14]:
print(cal_detector_to_world(x, y))
print(run_detector_to_world(x, y))

(80.49804757647831, -69.52285585022852)
(80.49805064896346, -69.52280678198792)


In [15]:
# What is the shift in the WCS before/after this step?
delta_ra = run_detector_to_world(x, y)[0] - cal_detector_to_world(x, y)[0]
delta_dec = run_detector_to_world(x, y)[1] - cal_detector_to_world(x, y)[1]
print('Shift in RA, Dec is ({:.4f}, {:.4f}) arcsec'.format(delta_ra * 3600., delta_dec * 3600.))
print('This is ({:.3f}, {:.3f}) pixels.'.format(delta_ra * 3600. / .062, delta_dec * 3600. / 0.062))

Shift in RA, Dec is (0.0111, 0.1766) arcsec
This is (0.178, 2.849) pixels.


The full GWCS model is contained in the ASDF extension of the file, and can be seen through the `meta` property. 

In [ ]:
tweak_data.meta.wcs

There are several world coordinate systems available in the file. 

In [17]:
# What coordinate frames are available?
tweak_data.meta.wcs.available_frames

['detector', 'v2v3', 'v2v3vacorr', 'v2v3corr', 'world']

In [18]:
# What is the input frame of the WCS object?
tweak_data.meta.wcs.input_frame

<Frame2D(name="detector", unit=(Unit("pix"), Unit("pix")), axes_names=('x', 'y'), axes_order=(0, 1))>

In [19]:
# What is the output frame of the WCS object?
tweak_data.meta.wcs.output_frame

<CelestialFrame(name="world", unit=(Unit("deg"), Unit("deg")), axes_names=('lon', 'lat'), axes_order=(0, 1), reference_frame=<ICRS Frame>)>

<a id='transforms'></a>
#### Do the coordinate transformations

Let's create a transformation function to go from detector pixels to location on the sky.

In [20]:
tweak_data.meta.wcs(50, 50)

(80.46244087451473, -69.54361944812793)

In [21]:
# Get the transform to go from detector to world coordinates
detector_to_world = tweak_data.meta.wcs.get_transform('detector', 'world')
pix_ra, pix_dec = detector_to_world(50, 50)
pix_ra, pix_dec

(80.46244087451473, -69.54361944812793)

And a function for the inverse transformation

In [22]:
world_to_detector = tweak_data.meta.wcs.get_transform('world', 'detector')
pix_x, pix_y = world_to_detector(pix_ra, pix_dec)
pix_x, pix_y

(50.01744404024873, 50.00591127827067)

<a id='find_coords'></a>
#### Determine positions using transforms

Using the transformation functions we defined above, we can now easily determine the RA and Dec of some randomly-chosen sources.

In [23]:
sources_x = [925., 945., 940.]
sources_y = [1045, 1183.2, 1120.5]

In [24]:
# Call the transform function
sources_ra, sources_dec = detector_to_world(sources_x, sources_y)

In [25]:
sources_ra

array([80.50026231, 80.50673304, 80.50374956])

In [26]:
sources_dec

array([-69.52439028, -69.52346722, -69.52381691])

And now the opposite case: My target is at a given RA and Dec, so where is it in this image?

In [27]:
targ_ra = 80.51
targ_dec = -69.53

In [28]:
# Call the inverse transform function
targ_x, targ_y = world_to_detector(targ_ra, targ_dec)

In [2]:
print('Target located at (x, y) = ({}, {})'.format(targ_x, targ_y))

NameError: name 'targ_x' is not defined

In [3]:
import jwst, tweakwcs
print(jwst.tweakreg.__file__)
print(tweakwcs.__file__)

/Users/arest/anaconda/envs/jwst/lib/python3.8/site-packages/jwst/tweakreg/__init__.py
/Users/arest/anaconda/envs/jwst/lib/python3.8/site-packages/tweakwcs/__init__.py
